***GENERATED CODE FOR testinterpret PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
from pyspark.ml.feature import StringIndexer
import json
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'transformation_label': 'Extract Date', 'feature_label': 'DateOfEntry'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
                                                'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'updatedLabel': 'DateOfEntry'}, {'transformation_label': 'Extract Date', 'feature_label': 'DateOfEntry'})
        transformationDF = transformationDF.drop('DateOfEntry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'PracticeCategory'}], 'feature': 'PracticeCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '98', 'mean': '', 'stddev': '', 'min': 'TPMG', 'max': 'TPMG', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeCategory'}, {'transformation_label': 'String Indexer', 'feature_label': 'PracticeCategory'})
        transformationDF = transformationDF.drop('PracticeCategory')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'transformation_label': 'Extract Date', 'feature_label': 'DateOfService'}], 'feature': 'DateOfService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'updatedLabel': 'DateOfService'}, {'transformation_label': 'Extract Date', 'feature_label': 'DateOfService'})
        transformationDF = transformationDF.drop('DateOfService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'CPTCategory'}], 'feature': 'CPTCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '98', 'mean': '80660.06', 'stddev': '21515.2', 'min': '15823', 'max': 'V5257', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCategory'}, {'transformation_label': 'String Indexer', 'feature_label': 'CPTCategory'})
        transformationDF = transformationDF.drop('CPTCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'DepartmentCategory'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '98', 'mean': '', 'stddev': '', 'min': '<Unspecified>', 'max': 'Ultrasound', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'transformation_label': 'String Indexer', 'feature_label': 'DepartmentCategory'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'FacilityCategory'}], 'feature': 'FacilityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '98', 'mean': '', 'stddev': '', 'min': 'Allergy And Asthma', 'max': 'Williamsburg Internal Medicine', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCategory'}, {'transformation_label': 'String Indexer', 'feature_label': 'FacilityCategory'})
        transformationDF = transformationDF.drop('FacilityCategory')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


def randomforestclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    rf_model = RandomForestClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    rf_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % rf_model.score(X_test, y_test))

    data = {'model': rf_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run testinterpretHooks.ipynb
try:
	#sourcePreExecutionHook()

	denyloan1 = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/demo/Denyloan1.csv', 'filename': 'Denyloan1.csv', 'token': '', 'delimiter': ','}")
	#sourcePostExecutionHook(denyloan1)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run testinterpretHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(denyloan1,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "98", "mean": "298.4", "stddev": "415.75", "min": "7", "max": "1800", "missing": "0"}, "updatedLabel": "Amount"}, {"transformationsData": [{"transformation_label": "Extract Date", "feature_label": "DateOfEntry"}], "feature": "DateOfEntry", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "DateOfEntry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "98", "mean": "-1.78", "stddev": "14.26", "min": "-67.21", "max": "75.48", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PrimaryInsuranceBal"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "PracticeCategory"}], "feature": "PracticeCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "98", "mean": "", "stddev": "", "min": "TPMG", "max": "TPMG", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeCategory"}, {"transformationsData": [{"transformation_label": "Extract Date", "feature_label": "DateOfService"}], "feature": "DateOfService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "DateOfService"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "CPTCategory"}], "feature": "CPTCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "98", "mean": "80660.06", "stddev": "21515.2", "min": "15823", "max": "V5257", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCategory"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "DepartmentCategory"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "98", "mean": "", "stddev": "", "min": "<Unspecified>", "max": "Ultrasound", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DiagnosisCodeID1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "98", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "DiagnosisCodeID1"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "FacilityCategory"}], "feature": "FacilityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "98", "mean": "", "stddev": "", "min": "Allergy And Asthma", "max": "Williamsburg Internal Medicine", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "98", "mean": "0.69", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "status"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run testinterpretHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=RandomForestClassifier(manualfe, ["Amount", "PrimaryInsuranceBal", "PracticeCategory_stringindexer", "CPTCategory_stringindexer", "DepartmentCategory_stringindexer", "DiagnosisCodeID1", "FacilityCategory_stringindexer"], "status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
